### IMPORTING LIBRARIES

In [87]:
import re
import nltk
import sys
import pandas as pd
import spacy
import gensim
from gensim.models import Word2Vec,FastText
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, ne_chunk
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [88]:
# Tokenization (splitting text into words/sentences)
nltk.data.path.append("C:/Users/DELL8/AppData/Roaming/nltk_data")
nltk.download('punkt', download_dir="C:/Users/DELL8/AppData/Roaming/nltk_data")


[nltk_data] Downloading package punkt to
[nltk_data]     C:/Users/DELL8/AppData/Roaming/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [89]:
nltk.download('stopwords')  # Removing common words (e.g., "the", "is", "and")
nltk.download('averaged_perceptron_tagger')  # POS tagging (identifying nouns, verbs, adjectives)
nltk.download('wordnet')  # WordNet database for synonyms and lemmatization
nltk.download('maxent_ne_chunker')  # Named Entity Recognition (NER) to identify names/locations
nltk.download('words')  # Dictionary of valid English words (used for spell checking, NER)
nltk.download('omw-1.4')  # Open Multilingual WordNet (support for multiple languages)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already

True

In [90]:
stop_words = set(stopwords.words('english'))

#### READ DATA

In [91]:
# Define file path
file_path = "D:\wiki file\enwiki-latest-abstract3.xml"

text_data = []
with open(file_path, "r", encoding="utf-8") as file:
    for _ in range(10000):  
        text_data.append(file.readline().strip())


#### Cleaning

In [92]:
import re
raw_text = " ".join(text_data)  
clean_text = re.sub(r"<.*?>", " ", raw_text)  
clean_text = re.sub(r"https?://\S+", " ", clean_text) 
clean_text = re.sub(r"[^a-zA-Z\s]", " ", clean_text)  




In [93]:


print("Raw Text (First 500 chars):", clean_text[:500])


Raw Text (First 500 chars):      Wikipedia  Diego Maradona stadium       Diego Maradona stadium can refer to       All article disambiguation pages        All disambiguation pages        Place name disambiguation pages        Short description is different from Wikidata             Wikipedia  White Stone       White Stone may refer to       See also             Wikipedia  Yes Tor         grid ref UK   SX            In popular culture        References        External links             Wikipedia  Watermelon Man  composition


#### Stopword Removal

#### Definition 
Stopword removal eliminates common words (e.g., "the," "and," "is") that add little meaning to text analysis, improving model efficiency. This helps focus on informative words, enhancing the accuracy of NLP tasks like vectorization and classification.


In [94]:
original_texts = []
stopword_removed_texts = []

with open(file_path, "r", encoding="utf-8") as file:
    for _ in range(10000):  
        line = file.readline().strip()

        clean_line = re.sub(r"<.*?>", " ", line)
        clean_line = re.sub(r"https?://\S+", " ", clean_line)
        clean_line = re.sub(r"[^a-zA-Z\s]", " ", clean_line)

        words = clean_line.lower().split()

        if not words:
            continue

        # Stopword Removal
        filtered_words = [word for word in words if word not in stop_words]
        
        original_texts.append(clean_line)
        stopword_removed_texts.append(" ".join(filtered_words))

In [95]:
# Create a DataFrame
df_stopwords = pd.DataFrame({"Original Text": original_texts, "Without Stopwords": stopword_removed_texts})
df_stopwords.head()


,Original Text,Without Stopwords
0,Wikipedia Diego Maradona stadium,wikipedia diego maradona stadium
1,Diego Maradona stadium can refer to,diego maradona stadium refer
2,All article disambiguation pages,article disambiguation pages
3,All disambiguation pages,disambiguation pages
4,Place name disambiguation pages,place name disambiguation pages


In [96]:
all_cleaned_text = " ".join(stopword_removed_texts)
word_counts = Counter(all_cleaned_text.split())
df_word_freq = pd.DataFrame(word_counts.items(), columns=["Word", "Score"]).sort_values(by="Score", ascending=False)
df_word_freq.head(10)

,Word,Score
0,wikipedia,732
27,references,616
28,external,404
29,links,404
18,also,237
17,see,213
240,history,150
195,career,122
8,place,115
425,birth,107


### Observation
Stopword removal filters out common words like **"the", "is", "in"**, making text **more meaningful and concise**. For example,  
 **Before:** *"The Diego Maradona stadium is used for international football matches."*  
 **After:** *"diego maradona stadium used international football"*  

This process **removes noise**, keeps **important words**, and improves **text analysis for NLP and machine learning**.

####  WordNet

#### **WordNet Definition:**  
WordNet is a lexical database for the English language that groups words into sets of synonyms called synsets, providing short definitions and usage examples. It captures relationships between words, such as synonyms, antonyms, hypernyms, and hyponyms, making it useful for natural language processing (NLP) tasks like text analysis, word similarity, and language modeling.

In [97]:
original_texts = []
wordnet_transformed = []

with open(file_path, "r", encoding="utf-8") as file:
    for _ in range(10000):  
        line = file.readline().strip()

        clean_line = re.sub(r"<.*?>", " ", line)
        clean_line = re.sub(r"https?://\S+", " ", clean_line)
        clean_line = re.sub(r"[^a-zA-Z\s]", " ", clean_line)

        words = clean_line.lower().split()

        if not words:
            continue

        synonyms = []
        for word in words:
            syns = wordnet.synsets(word)
            if syns:
                synonyms.append(syns[0].lemmas()[0].name()) 
            else:
                synonyms.append(word)


        original_texts.append(clean_line)
        wordnet_transformed.append(" ".join(synonyms))




In [98]:
df_wordnet = pd.DataFrame({"Original Text": original_texts, "WordNet Synonyms": wordnet_transformed})
df_wordnet.head(10)

,Original Text,WordNet Synonyms
0,Wikipedia Diego Maradona stadium,wikipedia diego maradona stadium
1,Diego Maradona stadium can refer to,diego maradona stadium can mention to
2,All article disambiguation pages,all article disambiguation page
3,All disambiguation pages,all disambiguation page
4,Place name disambiguation pages,topographic_point name disambiguation page
5,Short description is different from Wikidata...,short description be different from wikidata
6,Wikipedia White Stone,wikipedia White rock
7,White Stone may refer to,White rock May mention to
8,See also,see besides
9,Wikipedia Yes Tor,wikipedia yes tor


In [99]:
all_wordnet_text = " ".join(wordnet_transformed)
word_counts = Counter(all_wordnet_text.split())

df_wordnet_freq = pd.DataFrame(word_counts.items(), columns=["Word", "Score"]).sort_values(by="Score", ascending=False)
df_wordnet_freq.head(10)

,Word,Score
64,the,811
0,wikipedia,732
5,mention,667
72,of,603
105,and,433
34,links,404
33,external,404
30,inch,373
44,angstrom,321
15,be,272


#### Observation
The WordNet-based synonym replacement process in the code refines text preprocessing by standardizing word representations. It first cleans raw text by removing XML tags, URLs, and non-alphabetic characters, ensuring only meaningful words are processed. Each word is then mapped to its most relevant synonym using WordNet, replacing it when a synonym exists while retaining the original word if no match is found. This transformation reduces vocabulary complexity, enhances semantic consistency, and improves the performance of NLP and machine learning models in tasks like sentiment analysis, text classification, and search optimization by ensuring similar words are treated uniformly.

#### lemmatizer

##### **Lemmatizer Definition:**  
A **lemmatizer** is a tool in natural language processing (NLP) that reduces words to their base or dictionary form (lemma) by considering the word’s meaning and context. Unlike stemming, which simply removes suffixes, lemmatization ensures that the root word is a valid English word. It is commonly used in text preprocessing for machine learning and NLP applications.

In [100]:
lemmatizer = WordNetLemmatizer()
original_texts = []
lemmatized_texts = []

for line in text_data:
    clean_line = re.sub(r"<.*?>", " ", line)
    clean_line = re.sub(r"https?://\S+", " ", clean_line)
    clean_line = re.sub(r"[^a-zA-Z\s]", " ", clean_line)

    words = clean_line.lower().split()

    if not words:
        continue

    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    original_texts.append(clean_line)
    lemmatized_texts.append(" ".join(lemmatized_words))




In [101]:
df_lemmatized = pd.DataFrame({"Original Text": original_texts, "Lemmatized Text": lemmatized_texts})
df_lemmatized.head()

,Original Text,Lemmatized Text
0,Wikipedia Diego Maradona stadium,wikipedia diego maradona stadium
1,Diego Maradona stadium can refer to,diego maradona stadium can refer to
2,All article disambiguation pages,all article disambiguation page
3,All disambiguation pages,all disambiguation page
4,Place name disambiguation pages,place name disambiguation page


In [102]:
all_lemmatized_text = " ".join(lemmatized_texts)
word_counts = Counter(all_lemmatized_text.split())

df_lemmatized_freq = pd.DataFrame(word_counts.items(), columns=["Word", "Score"]).sort_values(by="Score", ascending=False)
df_lemmatized_freq.head(10)

,Word,Score
65,the,811
0,wikipedia,732
33,reference,618
73,of,603
106,and,433
35,link,409
34,external,404
45,a,392
30,in,372
15,is,250


In [103]:
lemmatized_full_text = " ".join(lemmatized_texts)
print("Lemmatized Text (First 500 chars):", lemmatized_full_text[:500])

Lemmatized Text (First 500 chars): wikipedia diego maradona stadium diego maradona stadium can refer to all article disambiguation page all disambiguation page place name disambiguation page short description is different from wikidata wikipedia white stone white stone may refer to see also wikipedia yes tor grid ref uk sx in popular culture reference external link wikipedia watermelon man composition length herbie hancock version mongo santamar a version chart performance herbie hancock version other version sample personnel ref


#### observation
Lemmatization significantly enhances text processing by converting words to their base or dictionary form while preserving their meanings. Unlike stemming, which often trims words without considering context, lemmatization ensures grammatical correctness. In the processed Wikipedia dataset, words like *running* become *run* and *better* becomes *good*, improving consistency for NLP tasks. The frequency distribution also changes, as different word variations merge into a single root form, reducing redundancy. This is crucial in machine learning applications like text classification and sentiment analysis, where standardized vocabulary enhances model accuracy and efficiency.

#### count_vectorizer

##### **Count Vectorizer Definition:**  
A **Count Vectorizer** is a technique in Natural Language Processing (NLP) that converts text data into a matrix of token counts. It represents each document as a vector, where each feature corresponds to a unique word, and the value represents the word's frequency in the document. This method is widely used in text classification, sentiment analysis, and other machine learning applications involving textual data.

In [104]:
cleaned_text = []
for line in text_data:
    clean_line = re.sub(r"<.*?>", " ", line)
    clean_line = re.sub(r"https?://\S+", " ", clean_line)
    clean_line = re.sub(r"[^a-zA-Z\s]", " ", clean_line)
    cleaned_text.append(clean_line)

final_clean_text = " ".join(cleaned_text)
count_vectorizer = CountVectorizer(max_features=10000)
count_vectors = count_vectorizer.fit_transform([final_clean_text])

feature_names = count_vectorizer.get_feature_names_out()
word_frequencies = count_vectors.toarray().flatten()
vectorized_df = pd.DataFrame(count_vectors.toarray(), columns=feature_names)


In [105]:
feature_names[:10]

array(['ab', 'abandoned', 'abandonment', 'abapeba', 'abba', 'abbreviated',
       'abc', 'abd', 'abduction', 'aberdeen'], dtype=object)

In [106]:
vectorized_df[:10]

,ab,abandoned,abandonment,abapeba,abba,abbreviated,abc,abd,abduction,aberdeen,...,zeus,ziffer,zip,zone,zong,zophorame,zophoryctes,zoua,zubayrids,zuleta
0,1,1,1,1,2,2,1,7,1,1,...,2,2,1,1,1,1,1,1,1,1


In [107]:
score_df = pd.DataFrame({"Word": feature_names, "score": word_frequencies})
score_df = score_df.sort_values(by="score", ascending=False)
score_df.head(10)

,Word,score
5200,the,811
5681,wikipedia,732
4329,references,616
3641,of,603
238,and,433
2973,links,404
1877,external,404
2540,in,372
2658,is,250
182,also,237


#### Observation
The Count Vectorizer converts cleaned text into a numerical format by counting word occurrences. It extracts the **top 10,000 most frequent words**, creating a matrix where each column represents a word and its frequency. The most common words are selected based on occurrence, making it useful for **text classification, topic modeling, and sentiment analysis** in machine learning.

### EXAMPLE 

In [108]:
sentences = [
    "Natural Language Processing is a subfield of AI.",
    "Machine learning models analyze and understand text.",
    "Count Vectorization converts text into numerical features.",
    "TF-IDF helps in identifying important words in a document.",
    "Word embeddings capture semantic meaning of words."
]
for i, sentence in enumerate(sentences):
    vectorizer = CountVectorizer() 
    X = vectorizer.fit_transform([sentence])  
    feature_names = vectorizer.get_feature_names_out()
    df_sentence = pd.DataFrame(X.toarray(), columns=feature_names)
    print(f"Sentence {i+1}: {sentence}")
    display(df_sentence) 

Sentence 1: Natural Language Processing is a subfield of AI.


,ai,is,language,natural,of,processing,subfield
0,1,1,1,1,1,1,1


Sentence 2: Machine learning models analyze and understand text.


,analyze,and,learning,machine,models,text,understand
0,1,1,1,1,1,1,1


Sentence 3: Count Vectorization converts text into numerical features.


,converts,count,features,into,numerical,text,vectorization
0,1,1,1,1,1,1,1


Sentence 4: TF-IDF helps in identifying important words in a document.


,document,helps,identifying,idf,important,in,tf,words
0,1,1,1,1,1,2,1,1


Sentence 5: Word embeddings capture semantic meaning of words.


,capture,embeddings,meaning,of,semantic,word,words
0,1,1,1,1,1,1,1


#### Tf-Idf_vectorizer

**TF-IDF Vectorizer:** It transforms text data into numerical representation by computing Term Frequency-Inverse Document Frequency (TF-IDF) scores, highlighting important words while reducing the weight of common words, making it useful for text mining and information retrieval.

In [111]:
cleaned_text = []
for line in text_data:
    clean_line = re.sub(r"<.*?>", " ", line) 
    clean_line = re.sub(r"https?://\S+", " ", clean_line)  
    clean_line = re.sub(r"[^a-zA-Z\s]", " ", clean_line)  
    cleaned_text.append(clean_line.lower())  

final_clean_text = " ".join(cleaned_text)

tfidf_vectorizer = TfidfVectorizer(max_features=10000)
tfidf_vectors = tfidf_vectorizer.fit_transform([final_clean_text])

feature_names = tfidf_vectorizer.get_feature_names_out()
word_frequencies = tfidf_vectors.toarray().flatten()
vectorized_df = pd.DataFrame(tfidf_vectors.toarray(), columns=feature_names)

In [112]:
feature_names[:10]

array(['ab', 'abandoned', 'abandonment', 'abapeba', 'abba', 'abbreviated',
       'abc', 'abd', 'abduction', 'aberdeen'], dtype=object)

In [113]:
vectorized_df

,ab,abandoned,abandonment,abapeba,abba,abbreviated,abc,abd,abduction,aberdeen,...,zeus,ziffer,zip,zone,zong,zophorame,zophoryctes,zoua,zubayrids,zuleta
0,0.000569,0.000569,0.000569,0.000569,0.001138,0.001138,0.000569,0.003984,0.000569,0.000569,...,0.001138,0.001138,0.000569,0.000569,0.000569,0.000569,0.000569,0.000569,0.000569,0.000569


In [114]:
value_df = pd.DataFrame({"Word": feature_names1, "score": word_frequencies})
value_df= score_df.sort_values(by="score", ascending=False)
value_df.head(10)

,Word,score
5200,the,811
5681,wikipedia,732
4329,references,616
3641,of,603
238,and,433
2973,links,404
1877,external,404
2540,in,372
2658,is,250
182,also,237


#### Observation 
The TF-IDF vectorization process transforms text data into numerical representations by weighting word importance based on term frequency (TF) and inverse document frequency (IDF). Unlike Count Vectorization, which represents raw word counts as integers, TF-IDF normalizes these counts, resulting in decimal values that highlight informative words while down-weighting common ones. If many words have similar TF-IDF scores, it indicates uniform distribution across the dataset. To improve variation, multiple documents should be used instead of a single merged text, and `max_df` and `min_df` parameters can help filter overly common or rare terms, enhancing feature significance for machine learning models.

In [115]:
sentences = [
    "Natural Language Processing is a subfield of AI.",
    "Machine learning models analyze and understand text.",
    "Count Vectorization converts text into numerical features.",
    "TF-IDF helps in identifying important words in a document.",
    "Word embeddings capture semantic meaning of words."
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
feature_names = vectorizer.get_feature_names_out()
df_tfidf = pd.DataFrame(X.toarray(), columns=feature_names)
for i, sentence in enumerate(sentences):
    words_in_sentence = set(sentence.lower().replace(".", "").split())  
    relevant_words = [word for word in words_in_sentence if word in feature_names]
    
    if not relevant_words:
        print(f"Sentence {i+1}: \"{sentence}\"")
        print("⚠ No words from this sentence found in the TF-IDF vocabulary.\n")
        continue
    filtered_df = df_tfidf.loc[[i], relevant_words]
    print(f"Sentence {i+1}: \"{sentence}\"")
    display(filtered_df) 


Sentence 1: "Natural Language Processing is a subfield of AI."


,of,language,processing,ai,is,subfield,natural
0,0.31284,0.387757,0.387757,0.387757,0.387757,0.387757,0.387757


Sentence 2: "Machine learning models analyze and understand text."


,learning,understand,machine,text,and,models,analyze
1,0.387757,0.387757,0.387757,0.31284,0.387757,0.387757,0.387757


Sentence 3: "Count Vectorization converts text into numerical features."


,converts,text,vectorization,count,numerical,into,features
2,0.387757,0.31284,0.387757,0.387757,0.387757,0.387757,0.387757


Sentence 4: "TF-IDF helps in identifying important words in a document."


,words,document,identifying,important,in,helps
3,0.247212,0.306413,0.306413,0.306413,0.612825,0.306413


Sentence 5: "Word embeddings capture semantic meaning of words."


,words,of,meaning,word,capture,embeddings,semantic
4,0.321388,0.321388,0.398352,0.398352,0.398352,0.398352,0.398352


#### **Word Frequency vs. TF-IDF**  

##### **🔹 Word Frequency (Count Vectorization)**
Word Frequency, also known as **Count Vectorization**, represents text by counting how many times each word appears in a document. It creates a matrix where rows correspond to documents, and columns represent unique words in the dataset. The values in the matrix are simply the number of times each word appears in a particular document. However, a major drawback of this approach is that it treats all words equally, meaning common words (e.g., "is," "the," "and") might dominate the representation, even if they don't carry meaningful information.

##### **🔹 TF-IDF (Term Frequency-Inverse Document Frequency)**
TF-IDF is an advanced weighting technique that improves upon simple word frequency. Instead of just counting occurrences, TF-IDF assigns importance to words based on how **unique** they are in a corpus. It consists of two parts:
1. **Term Frequency (TF)** – Measures how often a word appears in a document.
2. **Inverse Document Frequency (IDF)** – Reduces the weight of common words by assigning lower values to words that appear in many documents.

This method helps distinguish important words (such as "machine learning" or "AI") from less meaningful words (such as "is" or "a"), making it a more effective feature representation for text analysis, search engines, and machine learning models.

#### Word2Vec CBOW Model

##### **Define**:
A neural network-based word embedding technique that predicts a target word from its surrounding context, generating dense vectors that capture semantic relationships in text data.

In [116]:
cleaned_text = []
for line in text_data:
    clean_line = re.sub(r"<.*?>", " ", line)  
    clean_line = re.sub(r"https?://\S+", " ", clean_line)  
    clean_line = re.sub(r"[^a-zA-Z\s]", " ", clean_line) 
    cleaned_text.append(clean_line.lower())  

tokenized_sentences = [sentence.split() for sentence in cleaned_text if sentence]

word2vec_cbow = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=2, workers=4, sg=0)

word2vec_cbow.save("word2vec_cbow.model")

vocabulary_size = len(word2vec_cbow.wv)
vocabulary_words = list(word2vec_cbow.wv.index_to_key)[:10]


In [117]:
vocabulary_size 

2284

In [118]:
vocabulary_words[:10]

['the',
 'wikipedia',
 'references',
 'of',
 'and',
 'links',
 'external',
 'in',
 'a',
 'is']

In [119]:

word_to_check = "wikipedia"  # Choose a word from vocabulary
if word_to_check in word2vec_cbow.wv:
     similar_words = word2vec_cbow.wv.most_similar(word_to_check, topn=5)
     similar_words
else:
     word_to_check




In [120]:
similar_words

[('of', 0.9993835091590881),
 ('the', 0.9993541240692139),
 ('a', 0.9993164539337158),
 ('in', 0.9992851614952087),
 ('and', 0.9992839097976685)]

In [121]:
word_vectors = {word: word2vec_cbow.wv[word] for word in word2vec_cbow.wv.index_to_key}
df_word2vec = pd.DataFrame.from_dict(word_vectors, orient='index')

In [122]:
df_word2vec.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,-0.546425,0.396623,0.159661,0.569617,-0.030375,-1.085764,0.616014,1.336098,-0.520934,-0.385470,...,0.374024,-0.016361,0.216305,0.233187,0.946797,0.385969,-0.096585,-0.294352,0.058340,-0.082443
wikipedia,-0.236545,0.169883,0.070862,0.237698,-0.002081,-0.456449,0.255677,0.560029,-0.219355,-0.165310,...,0.157645,-0.006816,0.088463,0.089106,0.391321,0.162033,-0.030902,-0.121156,0.014148,-0.032210
references,-0.000321,0.003321,-0.006700,-0.001011,0.007652,0.006610,-0.003249,0.003545,-0.008677,0.005954,...,-0.004281,0.005718,0.009353,-0.003926,0.008592,0.005692,0.005867,0.000319,0.008275,-0.007082
of,-0.472527,0.345245,0.130968,0.473782,-0.011470,-0.918946,0.520205,1.134012,-0.434204,-0.332649,...,0.306168,-0.011210,0.179342,0.196689,0.811161,0.324481,-0.070464,-0.249764,0.046239,-0.081006
and,-0.373786,0.269551,0.096949,0.375487,-0.008498,-0.718305,0.410649,0.893572,-0.351425,-0.249061,...,0.253995,-0.016088,0.148301,0.153480,0.637560,0.248793,-0.053081,-0.185674,0.038221,-0.056452


#### Observation
The **Word2Vec CBOW model** efficiently learns word representations by predicting target words from surrounding context, mapping words to numerical vectors while preserving semantic meaning. When trained on Wikipedia data, it captures relationships between words, making it useful for **synonym detection, sentiment analysis, topic modeling, and recommendation systems**. Frequent words like "the" and "of" may have similar vectors due to shared contexts, revealing linguistic patterns that enhance **document similarity analysis and other NLP applications**.

### EXAMPLE

In [123]:
sentences = [
    "Natural Language Processing is a subfield of AI.",
    "Machine learning models analyze and understand text.",
    "Count Vectorization converts text into numerical features."
]
tokenized_sentences = [sentence.lower().replace(".", "").split() for sentence in sentences]
model = Word2Vec(sentences=tokenized_sentences, vector_size=5, window=2, min_count=1, sg=0)
word_vectors = model.wv

for i, sentence in enumerate(sentences):
    words_in_sentence = set(sentence.lower().replace(".", "").split())
    relevant_words = [word for word in words_in_sentence if word in word_vectors]
    if not relevant_words:
        print(f"Sentence {i+1}: \"{sentence}\"")
        print("⚠ No words from this sentence found in the Word2Vec vocabulary.\n")
        continue
    word_embeddings = {word: word_vectors[word] for word in relevant_words}
    df_embeddings = pd.DataFrame(word_embeddings).T

    print(f"Sentence {i+1}: \"{sentence}\"")
    display(df_embeddings)


Sentence 1: "Natural Language Processing is a subfield of AI."


,0,1,2,3,4
of,-0.192071,0.100146,-0.175192,-0.087837,-0.000702
language,-0.036320,0.057532,0.019837,-0.165704,-0.188976
processing,0.146236,0.101429,0.135124,0.015242,0.126989
ai,-0.005925,-0.153248,0.192324,0.099656,0.184691
is,-0.068107,-0.018928,0.115371,-0.150433,-0.078722
subfield,-0.038755,0.161549,-0.118618,0.000903,-0.095075
natural,-0.172394,0.073315,0.103798,0.114839,0.149338
a,-0.150232,-0.018601,0.190762,-0.146383,-0.046675


Sentence 2: "Machine learning models analyze and understand text."


,0,1,2,3,4
learning,0.147606,-0.030647,-0.090806,0.131066,-0.097275
understand,0.011491,0.148871,-0.016278,-0.052735,-0.175058
machine,-0.163061,0.090023,-0.082728,0.016480,0.170029
text,-0.010717,0.004727,0.102061,0.180197,-0.186074
and,0.109173,0.166940,-0.028973,-0.184150,0.087502
models,-0.089244,0.090350,-0.135739,-0.070970,0.187970
analyze,0.049425,-0.017693,0.110665,-0.054888,0.045233


Sentence 3: "Count Vectorization converts text into numerical features."


,0,1,2,3,4
converts,0.032630,0.003798,0.069473,0.004356,0.192377
text,-0.010717,0.004727,0.102061,0.180197,-0.186074
vectorization,0.037176,0.121777,-0.095961,-0.062145,0.135953
count,-0.017122,0.056540,0.108036,0.141052,-0.114048
numerical,-0.031553,0.006427,-0.082813,-0.153654,-0.030160
into,0.101212,-0.178348,-0.140831,0.018029,0.127851
features,-0.142336,0.129177,0.179460,-0.100309,-0.075267


### FastText Model

**Define:** An extension of Word2Vec that represents words as subword n-grams, allowing it to handle rare and misspelled words better. It is useful for tasks like text classification, word embeddings, and handling out-of-vocabulary words in NLP.

In [124]:
cleaned_text = []
for line in text_data:
    clean_line = re.sub(r"<.*?>", " ", line)  
    clean_line = re.sub(r"https?://\S+", " ", clean_line)  
    clean_line = re.sub(r"[^a-zA-Z\s]", " ", clean_line) 
    cleaned_text.append(clean_line.lower())  

tokenized_sentences = [sentence.split() for sentence in cleaned_text if sentence]
fasttext_model = FastText(sentences=tokenized_sentences, vector_size=100, window=5, min_count=2, workers=4)
fasttext_model.save("fasttext.model")

vocabulary_size = len(fasttext_model.wv)
vocabulary_words = list(fasttext_model.wv.index_to_key)[:10] 

In [125]:
vocabulary_size

2284

In [126]:
vocabulary_words

['the',
 'wikipedia',
 'references',
 'of',
 'and',
 'links',
 'external',
 'in',
 'a',
 'is']

In [127]:
word_to_check = "wikipedia"
if word_to_check in fasttext_model.wv:
     similar_words = fasttext_model.wv.most_similar(word_to_check, topn=5)
     similar_words
else:
     word_to_check

In [128]:
similar_words

[('international', 0.9999851584434509),
 ('national', 0.9999842047691345),
 ('station', 0.999983549118042),
 ('publication', 0.9999830722808838),
 ('expedition', 0.9999825358390808)]

In [129]:
word_vectors = {word: fasttext_model.wv[word] for word in fasttext_model.wv.index_to_key}
df_fasttext = pd.DataFrame.from_dict(word_vectors, orient='index')
df_fasttext.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,-0.513465,1.349513,-0.597473,0.361256,0.068596,0.089390,1.236055,0.178643,0.803634,-1.450158,...,-0.968439,0.009418,0.274101,0.531444,-0.744078,0.951133,-0.314721,-0.626773,0.088618,0.500517
wikipedia,-0.176461,0.465543,-0.204888,0.123062,0.023350,0.030601,0.423948,0.059704,0.276113,-0.501304,...,-0.334614,0.005342,0.093545,0.183711,-0.256214,0.328460,-0.110367,-0.214245,0.030502,0.172310
references,-0.118005,0.306832,-0.136185,0.083243,0.016504,0.021153,0.282786,0.042890,0.182171,-0.331251,...,-0.222837,0.003575,0.062819,0.122550,-0.171204,0.219704,-0.071241,-0.142994,0.019926,0.114243
of,-0.363401,0.959078,-0.424461,0.252435,0.056424,0.065014,0.871390,0.128798,0.573430,-1.026879,...,-0.689369,0.009981,0.196310,0.376713,-0.525070,0.675796,-0.223657,-0.444777,0.062501,0.350294
and,-0.395595,1.031444,-0.453087,0.277106,0.053592,0.072267,0.940852,0.133376,0.613616,-1.100832,...,-0.738742,0.005807,0.213177,0.408612,-0.563450,0.728726,-0.241735,-0.472973,0.067568,0.382662


#### observation
FastText is trained on tokenized sentences from Wikipedia data, creating dense word embeddings with **100-dimensional vectors**. The model considers **subword units (character n-grams)**, allowing it to handle out-of-vocabulary (OOV) words better than Word2Vec. The trained model is saved as `"fasttext.model"`, and key outputs include the **vocabulary size**, **sample words**, and **most similar words** to `"wikipedia"`. Since FastText captures morphological similarities, it helps in NLP tasks like **text classification, information retrieval, and named entity recognition (NER)** by generating meaningful word representations even for unseen words.

### Example 

In [130]:
sentences = [
    "Natural Language Processing is a subfield of AI.",
    "Machine learning models analyze and understand text.",
    "Count Vectorization converts text into numerical features."
]
tokenized_sentences = [sentence.lower().replace(".", "").split() for sentence in sentences]

model = FastText(sentences=tokenized_sentences, vector_size=5, window=2, min_count=1, sg=0)
word_vectors = model.wv
for i, sentence in enumerate(sentences):
    words_in_sentence = set(sentence.lower().replace(".", "").split())  
    relevant_words = [word for word in words_in_sentence if word in word_vectors]  
    if not relevant_words:
        print(f"Sentence {i+1}: \"{sentence}\"")
        print("⚠ No words from this sentence found in the FastText vocabulary.\n")
        continue
    word_embeddings = {word: word_vectors[word] for word in relevant_words}
    df_embeddings = pd.DataFrame(word_embeddings).T 
    df_embeddings.columns = [f"Sentence {i+1}" for _ in range(df_embeddings.shape[1])]

    print(f"Sentence {i+1}: \"{sentence}\"")
    display(df_embeddings) 


Sentence 1: "Natural Language Processing is a subfield of AI."


,Sentence 1,Sentence 1,Sentence 1,Sentence 1,Sentence 1
of,-0.100748,-0.058474,-0.064510,-0.059580,-0.047183
language,-0.000688,0.016565,-0.006263,-0.009926,-0.030405
processing,0.043831,0.009290,-0.023655,0.013634,0.013512
ai,-0.007351,-0.055547,0.105234,0.094669,-0.002168
is,-0.090077,-0.002271,0.056185,-0.071831,-0.048255
subfield,0.019183,-0.003308,0.000555,-0.052590,-0.001755
natural,-0.024299,0.037681,0.050791,-0.018423,0.031964
a,-0.124192,-0.061779,0.135200,-0.157351,-0.076799


Sentence 2: "Machine learning models analyze and understand text."


,Sentence 2,Sentence 2,Sentence 2,Sentence 2,Sentence 2
learning,-0.004460,0.008183,0.026833,-0.004881,-0.023256
understand,-0.009848,-0.010107,-0.002857,-0.041014,0.005478
machine,0.026910,0.042064,0.040274,0.006427,0.002441
text,0.017281,0.022135,0.007171,-0.000675,-0.046344
and,-0.021978,0.003003,0.010472,-0.072356,-0.030067
models,0.014962,0.047243,0.029115,0.037076,0.023462
analyze,0.011150,-0.016915,0.037830,-0.006047,-0.005326


Sentence 3: "Count Vectorization converts text into numerical features."


,Sentence 3,Sentence 3,Sentence 3,Sentence 3,Sentence 3
converts,-0.026735,0.032243,0.005513,0.000936,0.034568
text,0.017281,0.022135,0.007171,-0.000675,-0.046344
vectorization,0.016016,0.014197,-0.017473,0.016848,-0.014407
count,-0.035403,-0.027245,0.010775,-0.000303,0.021434
numerical,-0.005981,-0.006892,-0.000685,-0.005793,0.005259
into,0.047279,0.037817,0.028806,0.055436,0.069613
features,-0.012793,-0.017646,0.028540,0.025865,-0.026841
